<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Demo : Test  MOS Transistor

Version 1.0 (20/6/2018)

This **notebook** can be used to test **NMOS** transistors


## Import and connection

In [ ]:
# Import numpy
import numpy as np

# Import the main SLab module
import slab

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

In [ ]:
# Get the Vref value
vref = slab.gdata['vref']
vdd  = slab.gdata['vdd']

Execute the following **code cell** if you want the plots to be interactive.

In [ ]:
# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

## Basic NMOS connections

The following figure shows the basic connections to perform measurements on a **NMOS** Device Under Test (DUT).

![Base NMOS](images/Test_MOS/NMOS_Base.png)

The NPN bipolar transistor enables us to have higher currents than the ones available on the **DAC** outputs. Note that we are using a **BC547** transistor that is limited to a maxim $100 mA$ collector current. This is ok because you cannot drain much more current from a typical **SLab hardware board**. If you want to test a transistor at higher currents, you will need to use a more complex setup.

The $R_d$ resistor is used to measure the **drain** current. It will be selected for the maximum current we want to measure.

The **bias resistor** $R_b$ is included to provide a proper bias to the NPN emitter follower when the **NMOS** is draining no current. This resistor has no efect at all on the measurements so it can have any reasonable value as long as it is much bigger than $R_d$. In general $100 k\Omega$ is a good value.

## NMOS Saturation Curve

Draw the $I_D(V_{GS})$ saturation curve for a NMOS transistor

The $R_D$ resistance needs to be set for the maximum drain current to measure $I_{d\:Max}$.

We can obtain the voltages at **ADC1** and **ADC3**  

$\qquad V_{ADC1} = V_{DD}-V_{be}$

$\qquad V_{ADC2} = V_{DS}$

In order to be in the saturation region we need to verify:

$\qquad V_{DS} > V_{GS} - V_{TH}$

We can set this limit and obtain the needed resistance:

$\qquad R_d = \frac{V_{DD}-V_{be}-V_{GS\: Max}+V_{TH}}{I_{d\: Max}}$

If we set $V_{GS\: Max}$ to be $V_{DD}$ that simplifies to:

$\qquad R_d = \frac{V_{TH}-V_{be}}{I_{d\: Max}}$

The typical $V_{be}$ voltage of a NPN transistor is $0.7V$. That's ok because **NMOS** transistors usually have a $V_{TH}$ voltage higher than $0.7V$. For transistors with lower $V_{TH}$ we won't be able to use a $V_{GS}$ value as high as $V_{DD}$.

In the **code cell** below you need to indicate the $R_d$ value you are using. 

After executing it it gives the saturation $I_d(V_{DS})$ curve. The measurement stops when it detects that we are going out of the saturation region.

The code also indicates the $V_{GS}$ value that gets an $I_d$ current over $0.5mA$.

In [ ]:
# Set Rd value
Rd = 15

# Set nreadings
slab.setDCreadings(400)

# Vdac range
vdacR = np.arange(0,vref,0.1)

# Initialize output
vgsR = []
idR  = []

# Initialize Vth
vth = None

print('Measurement starts...')

# Perform measurement
slab.setVoltage(1,vref-0.1)
for vdac in vdacR:
    slab.setVoltage(2,vdac)           # Set gate voltage
    slab.wait(0.1)
    vgs = slab.readVoltage(2)         # Read gate voltage
    vrp = slab.readVoltage(1)         # Obtain Rd high node voltage
    vrn = slab.readVoltage(3)         # Obtain Rd low node voltage
    id = 1000.0*(vrp-vrn)/Rd          # Calculate Id (mA)
    if vth is None:
        # No Vth found yet. Try to find it
        if id>0.05:
            vth = vgs
            print('Vth (@Id=0.5mA) is about',vth,'V')
    else:
        # Vth is known. Check if we go out of saturation
        if vrn < (vgs-vth): 
            print('Getting out of saturation: Stop measuring')
            break
    # Store data        
    vgsR.append(vgs)        
    idR.append(id)

# Show graph
slab.plot11(vgsR,idR,'NMOS Saturation Curve','Vgs (V)','Id (mA)')

# Set DACs to zero
slab.zero()

## NMOS Drain Current Curves

Draw the $I_D(V_{DS})$ curves for several $V_{GS}$ values

This time we are not restricted to guarantee saturation, so the $R_d$ resistor just sets the current range you will get.

$\qquad I_d = \frac{V_{DD}-V_{be}-V_{DS}}{R_d}$

The current will be bounded by the limit at $V_{DS} = 0V$ give by:

$\qquad I_{d\: Max} = \frac{V_{DD}-V_{be}}{R_d}$

So, you can set Rd to be:

$\qquad R_d = \frac{V_{DD}-V_{be}}{I_{d\: Max}}$

But beware that you will only get this current if $V_{DS}$ is small

To run the **code below** you need to set, on the first lines:

* Minimum $V_{GS}$ to test
* Maximum $V_{GS}$ to test
* $V_{GS}$ increment between steps
* $R_d$ value

You can also change the **DAC1** sweep that sets the $V_{DS}$ voltage. That way you can change the resolution, for instance.

In [ ]:
# Parameters to set
Vgs_min  = 2.0    # [V]
Vgs_max  = 3.0    # [V]
Vgs_step = 0.25   # [V]
Rd       = 15     # [Ohm]

# Vdac1 sweep
Vdac1_min  = 0.5         # [V] 
Vdac1_max  = vref - 0.1  # [V]
Vdac1_step = 0.1         # [V]

# Set nreadings
slab.setDCreadings(400)

# Vgs range
vgsR = np.arange(Vgs_min,Vgs_max,Vgs_step)

# Initialize output lists
Vds_list = []
Id_list  = []
labels   = []
for vgs in vgsR:
    # Set Vgs
    slab.setVoltage(2,vgs)
    # Set label for this curve
    label = 'vgs = ' + str(int(100*slab.readVoltage(2))/100) + ' V'
    labels.append(label)
    # Perform dcSweep
    data = slab.dcSweep(1,Vdac1_min,Vdac1_max,Vdac1_step)
    # Calculate current (in mA)
    id = 1000.0*(data[1]-data[3])/Rd
    # Add to list
    Vds_list.append(data[3])
    Id_list.append(id)

# Draw all curves
slab.plotnn(Vds_list,Id_list,'Drain Current at several Vgs','Vds (V)','Id (mA)',labels)

You can use the code below to disconnect from the board when you are done

In [ ]:
# Disconnect from the board
slab.disconnect()

## Document license

Copyright  ©  Vicente Jiménez (2018)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">